In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
cd

C:\Users\37789


In [3]:
df = pd.read_csv("flightPrice.csv")

In [4]:
df

,Date_of_journey,Journey_day,Airline,Flight_code,Class,Source,Departure,Total_stops,Arrival,Destination,Duration_in_hours,Days_left,Fare
0,2023-01-16,Monday,SpiceJet,SG-8169,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5335
1,2023-01-16,Monday,Indigo,6E-2519,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.3333,1,5899
2,2023-01-16,Monday,GO FIRST,G8-354,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.1667,1,5801
3,2023-01-16,Monday,SpiceJet,SG-8709,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5794
4,2023-01-16,Monday,Air India,AI-805,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.1667,1,5955
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452083,2023-03-06,Monday,Vistara,UK-926,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,13.0833,50,65028
452084,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452085,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452086,2023-03-06,Monday,Vistara,UK-946,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,11.1667,50,72980


In [5]:
df.isnull().sum()

Date_of_journey      0
Journey_day          0
Airline              0
Flight_code          0
Class                0
Source               0
Departure            0
Total_stops          0
Arrival              0
Destination          0
Duration_in_hours    0
Days_left            0
Fare                 0
dtype: int64

In [6]:
df['Date_of_journey'] = pd.to_datetime(df['Date_of_journey'], format='%Y-%m-%d')
df["Day"] = df["Date_of_journey"].dt.day
df["Month"] = df["Date_of_journey"].dt.month
df["Year"] = df["Date_of_journey"].dt.year
df.drop(columns=["Date_of_journey"], inplace=True)

In [7]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

In [8]:
columns_to_encode = ['Class', 'Source','Airline','Journey_day','Destination']

In [9]:
one_hot_encoded_data = encoder.fit_transform(df[columns_to_encode])

In [10]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

one_hot_encoded_df.index = df.index

df.drop(columns=columns_to_encode, inplace=True)

df = pd.concat([df, one_hot_encoded_df], axis=1)


In [11]:
label_encoder = LabelEncoder()
df["Total_stops_encoded"] = label_encoder.fit_transform(df["Total_stops"])

df["Total_stops"] = df["Total_stops_encoded"]

In [12]:
df.drop(columns=["Flight_code"], inplace=True)

In [13]:
df["Departure"].unique()

array(['After 6 PM', 'Before 6 AM', '12 PM - 6 PM', '6 AM - 12 PM'],
      dtype=object)

In [14]:
df["Arrival"].unique()

array(['After 6 PM', 'Before 6 AM', '6 AM - 12 PM', '12 PM - 6 PM'],
      dtype=object)

In [15]:
Time_Slots = ['After 6 PM', 'Before 6 AM', '12 PM - 6 PM', '6 AM - 12 PM']

In [16]:
encoder = OneHotEncoder(categories=[Time_Slots,Time_Slots], drop='first', sparse_output=False)

In [17]:
one_hot_encoded_data = encoder.fit_transform(df[['Arrival', 'Departure']])

feature_names = encoder.get_feature_names_out(['Arrival', 'Departure'])

In [18]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data, columns=feature_names)
one_hot_encoded_df.index = df.index

In [19]:
df.drop(columns=['Arrival', 'Departure'], inplace=True)

In [20]:
df = pd.concat([df, one_hot_encoded_df], axis=1)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 43 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Total_stops             452088 non-null  int32  
 1   Duration_in_hours       452088 non-null  float64
 2   Days_left               452088 non-null  int64  
 3   Fare                    452088 non-null  int64  
 4   Day                     452088 non-null  int32  
 5   Month                   452088 non-null  int32  
 6   Year                    452088 non-null  int32  
 7   Class_Economy           452088 non-null  float64
 8   Class_First             452088 non-null  float64
 9   Class_Premium Economy   452088 non-null  float64
 10  Source_Bangalore        452088 non-null  float64
 11  Source_Chennai          452088 non-null  float64
 12  Source_Delhi            452088 non-null  float64
 13  Source_Hyderabad        452088 non-null  float64
 14  Source_Kolkata      

In [22]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [23]:
y = df['Fare']
X = df.drop(columns=['Fare'])

In [24]:
X = sm.add_constant(X)

In [25]:
model = sm.OLS(y, X).fit()

In [26]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                 6.385e+04
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        15:40:40   Log-Likelihood:            -4.6974e+06
No. Observations:              452088   AIC:                         9.395e+06
Df Residuals:                  452047   BIC:                         9.395e+06
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Total_stops            -1661

In [27]:
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

C:\Users\37789\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature          VIF
0              Total_stops          inf
1        Duration_in_hours     1.570360
2                Days_left   426.854725
3                      Day   176.346354
4                    Month   745.900733
5                     Year  4228.517933
6            Class_Economy     1.635916
7              Class_First     1.005208
8    Class_Premium Economy     1.511946
9         Source_Bangalore     2.306218
10          Source_Chennai     2.072468
11            Source_Delhi     2.454650
12        Source_Hyderabad     2.045468
13          Source_Kolkata     2.023243
14           Source_Mumbai     2.409069
15         Airline_AirAsia     1.189139
16        Airline_AkasaAir     1.072930
17     Airline_AllianceAir     1.011099
18        Airline_GO FIRST     1.135770
19          Airline_Indigo     1.835800
20        Airline_SpiceJet     1.060705
21         Airline_StarAir     1.002113
22         Airline_Vistara     1.693526
23      Journey_day_Monday     1.805282


In [28]:
# Extract p-values
p_values = model.pvalues

In [29]:
threshold = 0.05

In [30]:
features_above_threshold = p_values[p_values > threshold]

In [31]:
print("Features with p-value greater than 0.05:")
print(features_above_threshold)

Features with p-value greater than 0.05:
Airline_Indigo           0.079127
Airline_SpiceJet         0.063478
Journey_day_Monday       0.311049
Journey_day_Wednesday    0.592350
Destination_Delhi        0.944088
Arrival_12 PM - 6 PM     0.055597
dtype: float64


In [32]:
X = X.drop(columns=['Destination_Delhi'])
X = X.drop(columns=['Journey_day_Wednesday'])
X = X.drop(columns=['Journey_day_Monday'])
X = X.drop(columns=['Airline_Indigo'])
X = X.drop(columns=['Airline_SpiceJet'])
X = X.drop(columns=['Arrival_12 PM - 6 PM'])

In the earlier file we checked the model summary after dropping each value one by one, and we the results.
So to make the results simpler let's just drop all 6 values at once and then check the overall effect then head to the VIF values.

In [33]:
# Add a constant term to the feature matrix
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

In [34]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                 7.512e+04
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        15:42:37   Log-Likelihood:            -4.6974e+06
No. Observations:              452088   AIC:                         9.395e+06
Df Residuals:                  452053   BIC:                         9.395e+06
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Total_stops            -1662

Just like earlier we didnt see any changes in the R square value.

In [35]:
# Set a threshold for VIF
vif_threshold = 10

# Identify features with VIF greater than the threshold
features_above_threshold = vif_data[vif_data["VIF"] > vif_threshold]

# Print the features with VIF greater than the threshold
print(f"Features with VIF greater than {vif_threshold}:")
print(features_above_threshold)

Features with VIF greater than 10:
                Feature          VIF
0           Total_stops          inf
2             Days_left   426.854725
3                   Day   176.346354
4                 Month   745.900733
5                  Year  4228.517933
35  Total_stops_encoded          inf


In [36]:
# Check correlation between 'Total_stops' and 'Total_stops_encoded'
correlation_matrix = X[['Total_stops', 'Total_stops_encoded']].corr()
print("\nCorrelation between 'Total_stops' and 'Total_stops_encoded':")
print(correlation_matrix)


Correlation between 'Total_stops' and 'Total_stops_encoded':
                     Total_stops  Total_stops_encoded
Total_stops                  1.0                  1.0
Total_stops_encoded          1.0                  1.0


In [37]:
# Decide to drop 'Total_stops_encoded' based on correlation
X_dropped = X.drop(columns=['Total_stops_encoded'])

In [38]:
# Decide to drop 'Total_stops' based on correlation
X_dropped = X.drop(columns=['Total_stops'])

In [39]:
model_dropped = sm.OLS(y, X_dropped).fit()
print("\nModel summary after dropping 'Total_stops_encoded':")
print(model_dropped.summary())


Model summary after dropping 'Total_stops_encoded':
                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                 7.512e+04
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        15:42:39   Log-Likelihood:            -4.6974e+06
No. Observations:              452088   AIC:                         9.395e+06
Df Residuals:                  452053   BIC:                         9.395e+06
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [40]:
# Identify features with high VIF
high_vif_features = vif_data[vif_data["VIF"] > 10]["Feature"].tolist()

In [41]:
# Print features with high VIF values
print("\nFeatures with VIF greater than 10:")
print(vif_data[vif_data["VIF"] > 10])


Features with VIF greater than 10:
                Feature          VIF
0           Total_stops          inf
2             Days_left   426.854725
3                   Day   176.346354
4                 Month   745.900733
5                  Year  4228.517933
35  Total_stops_encoded          inf


C:\Users\37789\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [42]:
# Refit the model with the remaining features
model_dropped = sm.OLS(y, X_dropped).fit()

# Print the updated model summary
print("\nModel summary after dropping high VIF features:")
print(model_dropped.summary())



Model summary after dropping high VIF features:
                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                 7.512e+04
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        15:42:42   Log-Likelihood:            -4.6974e+06
No. Observations:              452088   AIC:                         9.395e+06
Df Residuals:                  452053   BIC:                         9.395e+06
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [43]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y, test_size=0.2, random_state=42)

In [50]:
# Assuming df is your DataFrame and 'Fare' is the target column
X = df.drop(columns=['Fare'])
y = df['Fare']

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Set a threshold for VIF
vif_threshold = 10

# Identify features with high VIF values
high_vif_features = vif_data[vif_data["VIF"] > vif_threshold]["Feature"]

# Drop features with high VIF values
X_dropped = X.drop(columns=high_vif_features)

# Recalculate VIF after dropping the features
vif_data_dropped = pd.DataFrame()
vif_data_dropped["Feature"] = X_dropped.columns
vif_data_dropped["VIF"] = [variance_inflation_factor(X_dropped.values, i) for i in range(X_dropped.shape[1])]

# Print the VIF data after dropping the features
print("\nVIF data after dropping high VIF features:")
print(vif_data_dropped)

# Refit the model with the remaining features
model_dropped = sm.OLS(y, X_dropped).fit()

# Print the updated model summary
print("\nModel summary after dropping high VIF features:")
print(model_dropped.summary())

# Performing RFE on the reduced set of features
model = LinearRegression()

# Initialize RFE with the linear regression model and choose the number of features to select
rfe = RFE(model, n_features_to_select=15)  # Adjust the number of features to select as needed
rfe = rfe.fit(X_dropped, y)

# Get the selected features
selected_features = X_dropped.columns[rfe.support_]

# Display the selected features
print("\nSelected Features: ", selected_features)

# Create a new feature set with the selected features
X_selected = X_dropped[selected_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Fit the linear regression model
model.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = model.predict(X_train)

# Make predictions on the test set
y_test_pred = model.predict(X_test)

# Evaluate the model on the training set
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Evaluate the model on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)


C:\Users\37789\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)



VIF data after dropping high VIF features:
                   Feature       VIF
0        Duration_in_hours  4.305941
1            Class_Economy  3.521669
2              Class_First  1.003576
3    Class_Premium Economy  1.780190
4         Source_Bangalore  2.164689
5           Source_Chennai  1.915864
6             Source_Delhi  2.298035
7         Source_Hyderabad  1.925023
8           Source_Kolkata  1.948995
9            Source_Mumbai  2.203176
10         Airline_AirAsia  1.227776
11        Airline_AkasaAir  1.046996
12     Airline_AllianceAir  1.008643
13        Airline_GO FIRST  1.148049
14          Airline_Indigo  2.025401
15        Airline_SpiceJet  1.069641
16         Airline_StarAir  1.002023
17         Airline_Vistara  3.141633
18      Journey_day_Monday  1.959150
19    Journey_day_Saturday  1.832119
20      Journey_day_Sunday  1.833667
21    Journey_day_Thursday  1.845260
22     Journey_day_Tuesday  1.841880
23   Journey_day_Wednesday  1.846447
24   Destination_Bangalore  2.1